# Importação de bibliotecas

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /spark-avro_2.11:4.0.0.jar pyspark-shell' 

In [2]:
# !git clone https://github.com/databricks/spark-avro.git

In [3]:
!cd spark-avro/

# Criação da Spark Session

In [4]:
spark2 = (SparkSession
         .builder
         .master('local')
         .appName('Salario-brasileiro')
         .config('spark.ui.port', '4050')
         .config('spark.jars.packages', 'com.databricks:apache-avro_2.11:4.0.0')
         .getOrCreate()
        )

In [5]:
spark2

# Extração dos arquivos

In [6]:
df_ipca_pib_full = spark2.read.parquet('gs://projeto_final_2021/pandas_to_parquet/IPCA_parquet/df_IPCA_PIB_full.snappy')

In [7]:
df_servidores = spark2.read.parquet('gs://projeto_final_2021/parquet_to_bq/df_serv2/*.parquet')

In [8]:
df_servidores.show(10)

+---+----+--------------------+--------------------+--------+--------------+------+
|mes| ano|                nome|               cargo|situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------+--------------+------+
|  1|2020|   AARAO LOPES COSTA|                CABO|   ATIVO|       4668.11|    MG|
|  1|2020|AARAO MARQUES DA ...|AGENTE DE SEGURAN...|   ATIVO|       4117.18|    MG|
|  1|2020|  AARON DUARTE DALLA|ESPEC.EM POLITICA...|   ATIVO|      16217.44|    MG|
|  1|2020|AARON FERRAREZ BO...|             SD 1 CL|   ATIVO|       3728.25|    MG|
|  1|2020|AARON FRANCA TEOFILO|PROFESSOR DE EDUC...|   ATIVO|       3875.45|    MG|
|  1|2020|ABADIA APARECIDA ...|AUXILIAR DE SERVI...|   ATIVO|       1338.45|    MG|
|  1|2020|ABADIA APARECIDA ...|AUXILIAR DE SERVI...|   ATIVO|       1384.61|    MG|
|  1|2020|ABADIA APARECIDA ...|PROFESSOR DE EDUC...|   ATIVO|       1438.16|    MG|
|  1|2020|ABADIA APARECIDA ...|AUXILIAR DE SERVI...|   ATIVO|       1384.61|

In [9]:
df_ipca_pib_full.show(10, truncate=11)

+---+----+------+-----------+---------+--------------+------------------+------------------+-------------------+---------------+------------+-----------------+
|mes| ano|estado|       item|IPCA_item|IPCA_VarMensal|IPCA_VarAcu_3Meses|IPCA_VarAcu_6Meses|IPCA_VarAcu_12Meses|IPCA_VarAcu_Ano|PIB_nacional|__index_level_0__|
+---+----+------+-----------+---------+--------------+------------------+------------------+-------------------+---------------+------------+-----------------+
|  1|2020|    MG|Índice_g...|      0.2|          0.21|              1.88|              2.05|               4.19|           0.21|    617747.2|                0|
|  1|2020|    MG|Alimenta...|      0.6|          0.21|              1.88|              2.05|               4.19|           0.21|    617747.2|                1|
|  1|2020|    MG|  Habitação|    -0.03|          0.21|              1.88|              2.05|               4.19|           0.21|    617747.2|                2|
|  1|2020|    MG|Artigos_...|     0.01| 

In [10]:
# 7_424_797 (inicial)
# 285_861_280 (on sem o estado)
# 71_465_320 (on com o estado)

In [11]:
# 74_247_970 (df_serv nova)

# Concatenação das tabelas

In [12]:
df_salario = df_servidores.join(df_ipca_pib_full, on=['mes','ano', 'estado'])

In [13]:
df_salario = df_salario.select('mes',
                               'ano',
                               'estado',
                               'nome',
                               'cargo',
                               'situacao',
                               'salarioLiquido',
                               'item',
                               'IPCA_item',
                               'IPCA_VarMensal',
                               'IPCA_VarAcu_3Meses',
                               'IPCA_VarAcu_6Meses',
                               'IPCA_VarAcu_12Meses',
                               'IPCA_VarAcu_Ano',
                               'PIB_nacional')

In [14]:
df_salario.show()

+---+----+------+--------------------+--------------------+--------+--------------+--------------------+---------+--------------+------------------+------------------+-------------------+---------------+------------+
|mes| ano|estado|                nome|               cargo|situacao|salarioLiquido|                item|IPCA_item|IPCA_VarMensal|IPCA_VarAcu_3Meses|IPCA_VarAcu_6Meses|IPCA_VarAcu_12Meses|IPCA_VarAcu_Ano|PIB_nacional|
+---+----+------+--------------------+--------------------+--------+--------------+--------------------+---------+--------------+------------------+------------------+-------------------+---------------+------------+
|  1|2020|    MG|   AARAO LOPES COSTA|                CABO|   ATIVO|       4668.11|         Comunicação|     0.24|          0.21|              1.88|              2.05|               4.19|           0.21|    617747.2|
|  1|2020|    MG|   AARAO LOPES COSTA|                CABO|   ATIVO|       4668.11|            Educação|     0.31|          0.21|   

In [15]:
# ./bin/spark-submit --packages org.apache.spark:spark-avro_2.12:3.1.2 ...

In [16]:
df_salario.write.format("avro").option('compression', 'snappy').save('gs://projeto_final_2021/parquet_to_bq/df_salario.avro')

AnalysisException: Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of "Apache Avro Data Source Guide".